In [ ]:
import pandas as pd
from transformers import pipeline
import json


def generate_income_statement(csv_file):   

    # error handling
    try:
        df = pd.read_csv(csv_file)
    except FileNotFoundError:
        return {"error": "CSV file not found."}
    except pd.errors.EmptyDataError:
        return {"error": "CSV file is empty."}
    except Exception as e:
        return {"error": f"An error occurred while reading the CSV: {e}"}

    # Initialize a text classification pipeline 
    classifier = pipeline("text-classification", model="distilbert-base-uncased")

    # categorize transactions
    def categorize_transaction(description):
        """Categorizes a transaction description."""
        description_lower = description.lower()
        if "sale" in description_lower or "order" in description_lower or "revenue" in description_lower:
            return "Revenue"
        elif "fee" in description_lower or "cost" in description_lower or "expense" in description_lower:
            return "Operating Expenses"
        else:
            return "Other"

    # Apply categorization 
    df["Category"] = df["Description"].apply(categorize_transaction)

    # Calculate sums for each category
    income_statement = {
        "Revenue": df[df["Category"] == "Revenue"]["Amount"].sum(),
        "Operating Expenses": df[df["Category"] == "Operating Expenses"]["Amount"].sum(),
    }

    # Calculate Net Income
    income_statement["Net Income"] = income_statement["Revenue"] - income_statement["Operating Expenses"]

    return json.dumps(income_statement, indent=4)


In [ ]:


# Generate income statement
income_statement_json = generate_income_statement("eco_bloom_transactions.csv")
print(income_statement_json)